In [ ]:
%pip install pytelegrambotapi
import telebot

Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 23.0.1 -> 25.1.1
[notice] To update, run: python3 -m pip install --upgrade pip


In [ ]:
import os
import json
import requests
import logging
import fitz  # PyMuPDF
import docx  # python-docx
import re  # Добавляем модуль для регулярных выражений

# URL to access the model
url = "https://llm.api.cloud.yandex.net/foundationModels/v1/completion"
file_path = "/home/jupyter/datasphere/project/test.docx"

# Настройка логирования
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)

class TextExtractor:
    def __init__(self, optimize=True):
        """
        Инициализация экстрактора текста
        :param optimize: Флаг оптимизации текста (нормализация пробелов)
        """
        self.optimize = optimize
        logger.info(f"TextExtractor инициализирован. Оптимизация текста: {'включена' if optimize else 'отключена'}")

    def extract_text(self, file_path: str) -> str:
        """Основной метод извлечения текста"""
        if not os.path.exists(file_path):
            raise FileNotFoundError(f"Файл не найден: {file_path}")

        ext = os.path.splitext(file_path)[1].lower()
        logger.info(f"Обработка файла {file_path} (тип: {ext})")

        try:
            if ext == '.pdf':
                text = self._extract_from_pdf(file_path)
            elif ext == '.docx':
                text = self._extract_from_docx(file_path)
            elif ext == '.txt':
                text = self._extract_from_txt(file_path)
            else:
                raise ValueError(f"Неподдерживаемый формат файла: {ext}")

            # Применяем оптимизацию текста, если включена
            if self.optimize:
                text = self._optimize_text(text)

            # Логируем статистику по тексту
            original_length = len(text)
            logger.info(f"Извлечено текста: {original_length} символов")

            return text
        except Exception as e:
            logger.error(f"Ошибка при обработке файла: {str(e)}")
            raise

    def _extract_from_pdf(self, pdf_path: str) -> str:
        """Извлечение текста из PDF с помощью PyMuPDF"""
        logger.info("Извлечение текста из PDF...")
        text = ""
        try:
            with fitz.open(pdf_path) as doc:
                for page in doc:
                    # Извлекаем текст страницы
                    page_text = page.get_text()
                    if page_text.strip():  # Проверяем, есть ли текст на странице
                        text += page_text + "\n\n"
            return text
        except Exception as e:
            logger.error(f"Ошибка обработки PDF: {str(e)}")
            raise RuntimeError(f"Не удалось обработать PDF: {str(e)}")

    def _extract_from_docx(self, docx_path: str) -> str:
        """Извлечение текста из DOCX с помощью python-docx"""
        logger.info("Извлечение текста из DOCX...")
        try:
            doc = docx.Document(docx_path)
            full_text = []
            for para in doc.paragraphs:
                full_text.append(para.text)

            # Обработка таблиц
            for table in doc.tables:
                for row in table.rows:
                    for cell in row.cells:
                        full_text.append(cell.text)

            return "\n".join(full_text)
        except Exception as e:
            logger.error(f"Ошибка обработки DOCX: {str(e)}")
            raise RuntimeError(f"Не удалось обработать DOCX: {str(e)}")

        def _extract_from_txt(self, txt_path: str) -> str:
            # Чтение текстовых файлов с обработкой кодировок
            logger.info("Извлечение текста из TXT...")
            encodings = ['utf-8', 'windows-1251', 'cp1251', 'iso-8859-5', 'koi8-r']
            for encoding in encodings:
                try:
                    with open(txt_path, 'r', encoding=encoding) as f:
                        return f.read()
                except UnicodeDecodeError:
                    continue
                except Exception as e:
                    logger.error(f"Ошибка чтения TXT ({encoding}): {str(e)}")
                    continue
            raise RuntimeError(f"Не удалось декодировать файл {txt_path} с использованием стандартных кодировок")

    def _optimize_text(self, text: str) -> str:
        """
        Оптимизация текста:
        1. Замена последовательностей пробельных символов на один пробел
        2. Удаление пробелов в начале и конце текста
        """
        # Заменяем все последовательности пробелов, табуляций, переносов на один пробел
        optimized = re.sub(r'\s+', ' ', text)

        # Удаляем пробелы по краям
        optimized = optimized.strip()

        # Логируем изменения
        original_len = len(text)
        optimized_len = len(optimized)
        reduction = original_len - optimized_len

        if reduction > 0:
            logger.info(f"Оптимизация текста: удалено {reduction} лишних символов "
                        f"({reduction/original_len:.1%} сокращение)")

        return optimized


In [ ]:
import pypandoc
import os

def generate_docx(json, topic):
    test_data = json["questions"]  # Ваш JSON-массив с вопросами
    markdown_test = json_to_markdown(test_data, topic)
    with open("tmp.md", "w", encoding="utf-8") as f:
        f.write(markdown_test)
    outputfile = f"/home/jupyter/datasphere/project/generated_tests/{topic}.docx"
    output = pypandoc.convert_file('tmp.md', 'docx', outputfile=outputfile)
    os.remove("tmp.md")
    return outputfile

In [ ]:
%pip install import_ipynb

Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 23.0.1 -> 25.1.1
[notice] To update, run: python3 -m pip install --upgrade pip


In [ ]:
import telebot
from telebot import types
from telebot.types import ReplyKeyboardMarkup, KeyboardButton
import os

num_questions=-1
topic=''
%store num_questions
%store topic

STATE=0
TEACHERS_FOLDER = "/home/jupyter/datasphere/project/teachers files"
chosen_doc=""

bot = telebot.TeleBot('your_token')
def get_yes_no():
    markup = ReplyKeyboardMarkup(row_width=2, resize_keyboard=True)
    btn1 = KeyboardButton('Да')
    btn2 = KeyboardButton('Нет')
    markup.add(btn1, btn2)
    return markup

def get_file_names():
    markup = ReplyKeyboardMarkup(row_width=2, resize_keyboard=True)

    # Add buttons
    for file_name in os.listdir(TEACHERS_FOLDER):
        if not file_name.startswith('.'):
            btn = KeyboardButton(file_name)
            markup.add(btn)
    return markup

@bot.message_handler(commands=["start"])
def start(m, res=False):
    bot.send_message(m.chat.id, 'Бот запущен. Загрузите документы, по которым необходимо составить тесты. Название каждого документа должно отражать его истинное содержание. '
                                 +'После загрузки всех документов документов отправьте команду "/done"')

@bot.message_handler(commands=["done"])
def done(message, res=False):
    global STATE
    STATE=1
    bot.send_message(message.chat.id, f"Загрузка файлов завершена. Введите количество вопросов одним числом:")

@bot.message_handler(content_types=["document"])
def handle_document(message):
    #сделать рекурсию для загрузки нескольких доков
    file_info = bot.get_file(message.document.file_id)
    downloaded_file = bot.download_file(file_info.file_path)
        #сохраняем файл
    file_name = message.document.file_name
    save_path = os.path.join(TEACHERS_FOLDER, file_name)

    with open(save_path, 'wb') as new_file:
        new_file.write(downloaded_file)

@bot.message_handler(content_types=["text"])
def handle_text(message):
    '''
    if STATE==1:
        topic=message.text
        %store topic
        bot.send_message(message.chat.id, 'Тема сохранена.')
        STATE=2
    '''
    global STATE
    global num_questions

    if STATE==1:
        num_questions=int(message.text)
        %store num_questions
        bot.send_message(message.chat.id, f'Сгенерировать тест для скачивания ({num_questions} вопросов, формат .docx) по одному из загруженных документов?', reply_markup=get_yes_no())
        STATE=2
    if STATE==2:
        if message.text=='Да':
            bot.send_message(message.chat.id, "Выберите документ.",reply_markup=get_file_names())
            STATE=3
        elif message.text=='Нет':
            bot.send_message(message.chat.id, 'Тесты готовы к прохождению в @TestoryBot!')
    elif STATE==3 and ('.txt' in message.text or '.pdf' in message.text or '.docx' in message.text):
        #ГЕНЕРАЦИЯ DOCX
        global chosen_doc
        chosen_doc=message.text
        %store chosen_doc
        print("chosen doc ", chosen_doc)
        g_t=generate_test(chosen_doc, num_questions)
        print("g_t ", g_t)
        g_d=generate_docx(g_t, chosen_doc.rsplit('.', 1)[0])
        print("type(g_d)", type(g_d))
        bot.send_document(
            message.chat.id,
            document=open(g_d, 'rb'),
            caption="Сгенерированный тест по теме: ")
        bot.send_message(message.chat.id, "Благодарим за использование Testory! Для повтора нажмите /start")
        #ветка с кнопками да (тогда выбираем из списка документов и генерим docx)\нет


Stored 'num_questions' (int)
Stored 'topic' (str)


In [ ]:
def json_to_markdown(json_data, topic) -> str:
    markdown = [
        "# Тест по теме: " + topic + "\n",
        "**ФИО:** _____________________________________________________________\n"
    ]

    for item in json_data:
        # Заголовок вопроса
        markdown.append(f"\n### {item['number']}. {item['question']}\n")

        # Варианты ответов
        for letter, text in item['options'].items():
            markdown.append(f" **{letter}.** {text}\n")

        # Разделитель после вопроса
        #markdown.append("\n---")

    return "\n".join(markdown)

In [ ]:
def parse_text(text):
    blocks = text.strip().split("EOQ")
    questions = []

    for block in blocks:
        block = block.strip()
        if not block:
            continue
        if block.startswith("BOQ"):
            block = block[3:].strip()

        lines = [line.strip() for line in block.splitlines() if line.strip()]
        if not lines:
            continue

        # Обработка номера и вопроса
        first_line = lines[0]
        if '.' not in first_line:
            continue
        num_part, q_text = first_line.split('.', 1)
        try:
            num = int(num_part.strip())
        except:
            num = num_part.strip()
        question_text = q_text.strip()

        # Извлечение вариантов ответов
        options = {}
        idx = 1
        expected_letters = ['A', 'B', 'C', 'D']

        for letter in expected_letters:
            if idx >= len(lines):
                break
            line = lines[idx]
            if line.startswith(f"{letter})"):
                option_text = line.split(')', 1)[1].strip()
                options[letter] = option_text
                idx += 1
            else:
                break

        # Поиск правильного ответа
        answer = None
        while idx < len(lines):
            if lines[idx].startswith("Answer:"):
                ans_part = lines[idx].split(':', 1)[1].strip()
                if ans_part in expected_letters:
                    answer = ans_part
                idx += 1
                break
            idx += 1

        # Извлечение объяснения
        explanation_lines = []
        while idx < len(lines):
            if lines[idx].startswith("Explanation:"):
                expl_part = lines[idx].split(':', 1)[1].strip()
                if expl_part:
                    explanation_lines.append(expl_part)
                idx += 1
                break
            idx += 1

        while idx < len(lines):
            explanation_lines.append(lines[idx])
            idx += 1

        explanation_text = "\n".join(explanation_lines)

        # Формирование словаря вопроса
        question_dict = {
            "number": num,
            "question": question_text,
            "options": options,
            "answer": answer,
            "explanation": explanation_text
        }
        questions.append(question_dict)

    return {"questions": questions}

In [ ]:
extracted_text=''
token='your_token'
def generate_test(file_name, num):
    global num_questions
    num_questions = num
    # ?поменять ф-ию, чтобы она принимала путь к файлу любого типа
    UPLOAD_FOLDER = "/home/jupyter/datasphere/project/teachers files"
    path = os.path.join(UPLOAD_FOLDER, file_name)
    extractor = TextExtractor()
    extracted_text = extractor.extract_text(path)

    # Формирование системного промпта с содержимым файла
    system_prompt = f"""**Роль:** Ты — опытный преподаватель и эксперт по предмету, представленному в текстовом документе. Твоя задача — создать содержательный тест для проверки глубокого понимания ключевых аспектов документа.

    **Инструкция:**

    1.  **Тщательный анализ:** Внимательно прочти и проанализируй предоставленный ниже текстовый документ.
    2.  **Выделение сути:** Определи {num_questions} *самых важных и существенных* тем, концепций, аргументов, принципов или фактов, которые являются **основополагающими** для понимания всего документа. Сосредоточься на глубине, а не на количестве деталей. Избегай тривиальных или малозначимых моментов.
    3.  **Генерация теста:** На основе выделенных ключевых моментов создай **ровно {num_questions} вопросов** с множественным выбором (multiple choice). Каждый вопрос должен иметь **ровно 4 варианта ответа** и **ровно 1 правильный ответ**.
        *   **Качество вопросов:**
            *   Каждый вопрос должен напрямую относиться к **одному из выявленных важных моментов**.
            *   Вопросы должны проверять **понимание сути, концепций, взаимосвязей или критических выводов**, а не просто механическое запоминание мелких деталей.
            *   Формулируй вопросы четко, однозначно и грамотно.
        *   **Варианты ответов:**
            *   Только **один** вариант должен быть абсолютно правильным.
            *   Остальные три варианта (дистракторы) должны быть **правдоподобными, но ошибочными**. Они должны отражать типичные заблуждения, неполное понимание или логические ошибки, которые может допустить человек, не до конца усвоивший материал.
            *   Избегай дистракторов, которые явно нелепы или не имеют отношения к вопросу.
    4.  **Формат вывода:**
        *   Выведи **только** готовый тест.
        *   Пронумеруй вопросы от 1 до {num_questions}.
        *   Для каждого вопроса перечисли 4 варианта ответа, обозначив их буквами **A, B, C, D**. Это очень важно.
        *   Приведи правильные варианты ответов
        *   Сразу начни генерировать тест, не нужно приводить никакие приветственные фразы
        *   Я приведу шаблон, которому ты должен строго следовать:
        **Шаблон:**
        BOQ
        <question number>. <question>
        A) <option 1>
        B) <option 2>
        C) <option 3>
        D) <option 4>

        Answer:
        Explanation:
        EOQ

        **Пример теста, соответствующий шаблону:**
        BOQ
        1. Кто должен проходить обучение в аккредитованной организации по охране труда?
        A) Все работники без исключения.
        B) Только руководители и специалисты по охране труда.
        C) Минимум работников, зависящий от среднесписочной численности компании и категории риска.
        D) Только работники, занятые на работах повышенной опасности.

        Answer: C

        Explanation:
        Ответ A [неверен/верен], потому что [обоснование со ссылкой на НПА].
        Ответ B [неверен/верен], потому что [обоснование со ссылкой на НПА].
        Ответ C [неверен/верен], потому что [обоснование со ссылкой на НПА].
        Ответ D [неверен/верен], потому что [обоснование со ссылкой на НПА].
        EOQ

        BOQ
        2. Как часто проводится плановое обучение по охране труда для работников?
        A) Раз в год.
        B) Раз в три года.
        C) Раз в пять лет.
        D) По мере необходимости.

        Answer: B

        Explanation:
        Ответ A [неверен/верен], потому что [обоснование со ссылкой на НПА].
        Ответ B [неверен/верен], потому что [обоснование со ссылкой на НПА].
        Ответ C [неверен/верен], потому что [обоснование со ссылкой на НПА].
        Ответ D [неверен/верен], потому что [обоснование со ссылкой на НПА].
        EOQ

        ....

        **В шаблоне BOQ означает "beginning of question", EOQ означает "end of question"

    **Текст документа для анализа:**
    {extracted_text}

    """

    # Подготовка данных запроса
    data = {
        "modelUri": "gpt://b1ghseujph70d1blspqm/yandexgpt",
        "completionOptions": {
            "stream": False,
            "temperature": 0.0,
            "maxTokens": 5000
        },
        "messages": [
            {
                "role": "system",
                "text": system_prompt
            },
            {
                "role": "user",
                "text": "Сгенерируй тест на основе предоставленного документа."
            }
        ]
    }


    # Получение ответа от модели (убедитесь, что переменная token определена)
    response = requests.post(
        url,
        headers={"Authorization": "Bearer " + token},
        json=data
    ).json()

    # Извлечение и вывод результата
    try:
        generated_text = response['result']['alternatives'][0]['message']['text']
        return parse_text(generated_text)
        #print(generated_text)
    except KeyError:
        print("Ошибка в ответе API:")
        print(response)

In [ ]:
%pip install pypandoc_binary

Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 23.0.1 -> 25.1.1
[notice] To update, run: python3 -m pip install --upgrade pip


In [ ]:
import pypandoc

In [ ]:
bot.polling(none_stop=True, interval=0)

Stored 'num_questions' (int)
Stored 'num_questions' (int)
Stored 'num_questions' (int)
Stored 'num_questions' (int)


2025-07-12 08:31:58,766 - INFO - TextExtractor инициализирован. Оптимизация текста: включена
2025-07-12 08:31:58,767 - INFO - Обработка файла /home/jupyter/datasphere/project/teachers files/Типовая_ситуация_Обучение_по_охране_труда_основные_правила.txt (тип: .txt)
2025-07-12 08:31:58,769 - ERROR - Ошибка при обработке файла: 'TextExtractor' object has no attribute '_extract_from_txt'


Stored 'chosen_doc' (str)
chosen doc  Типовая_ситуация_Обучение_по_охране_труда_основные_правила.txt


AttributeError: 'TextExtractor' object has no attribute '_extract_from_txt'

In [ ]:
STATE

2